[View in Colaboratory](https://colab.research.google.com/github/ylongqi/openrec/blob/master/tutorials/Youtube_Recommender_example.ipynb)

<p align="center">
  <img src ="https://recsys.acm.org/wp-content/uploads/2017/07/recsys-18-small.png" height="40" /> <font size="4">Recsys 2018 Tutorial</font>
</p>
<p align="center">
  <font size="4"><b>Modularizing Deep Neural Network-Inspired Recommendation Algorithms</b></font>
</p>
<p align="center">
  <font size="4">Hands on: Customizing Deep YouTube Video Recommendation. Youtube example</font>
</p>

# the Youtube Recommender

To implement  a model using OpenRec, you will need to first decide how this recommender should be decomposed into subgraphs, i.e., inputgraph, usergraph, itemgraph, interactiongraph and optimizergraph. For example, the training graph of YouTube-Rec can be decomposed as follows.

<p align="center">
  <img src ="https://s3.amazonaws.com/cornell-tech-sdl-openrec/tutorials/youtube_rec_module.png" height="400" />
</p>




* **inputgraph**: user demographis, item consumption history and the groundtruth label.
* **usergraph**: extract user-specific latent factor.
* **itemgraph**: extract latent factors for items.
* **interactiongraph**: uses MLP and softmax to model user-item interactions.

After defining subgraphs, their interfaces and connections need to be specified. A sample specification of YouTube-Rec can be as follows.
<p align="center">
  <img src ="https://s3.amazonaws.com/cornell-tech-sdl-openrec/tutorials/youtube_rec.png" height="300" />
</p>

# Install OpenRec and download dataset

In [0]:
!pip install openrec

import urllib.request

dataset_prefix = 'http://s3.amazonaws.com/cornell-tech-sdl-openrec'
urllib.request.urlretrieve('%s/lastfm/lastfm_test.npy' % dataset_prefix, 
                   'lastfm_test.npy')
urllib.request.urlretrieve('%s/lastfm/lastfm_train.npy' % dataset_prefix, 
                   'lastfm_train.npy')
urllib.request.urlretrieve('%s/lastfm/user_feature.npy' % dataset_prefix, 
                   'user_feature.npy')

In [0]:
import numpy as np
import random
from openrec.utils.samplers import Sampler

def DRRSampler(dataset, batch_size, max_seq_len, user_feature, num_process=5, seed=100, sort=True):

    random.seed(seed)
    def batch(dataset, user_feature=user_feature, max_seq_len=max_seq_len, batch_size=batch_size):

        while True:
            input_npy = np.zeros(batch_size, dtype=[('seq_item_id', (np.int32,  max_seq_len)),
                                                   ('seq_len', np.int32),
                                                   ('label', np.int32),
                                                   ('user_gender', np.int32),
                                                   ('user_geo', np.int32)])

            for ind in range(batch_size):
                user_id = random.randint(0, dataset.total_users()-1)
                item_list = dataset.get_positive_items(user_id, sort=sort)
                while len(item_list) <= 1:
                    user_id = random.randint(0, dataset.total_users()-1)
                    item_list = dataset.get_positive_items(user_id, sort=sort)
                predict_pos = random.randint(1, len(item_list) - 1)
                train_items = item_list[max(0, predict_pos-max_seq_len):predict_pos]
                pad_train_items = np.zeros(max_seq_len, np.int32)
                pad_train_items[:len(train_items)] = train_items
                input_npy[ind] = (pad_train_items,
                                  len(train_items),
                                  item_list[predict_pos],
                                  user_feature[user_id]['user_gender'],
                                  user_feature[user_id]['user_geo'])
            yield input_npy

    s = Sampler(dataset=dataset, generate_batch=batch, num_process=num_process)

    return s
  
  


def DRREvaluationSampler(dataset, max_seq_len, user_feature, seed=100, sort=True):

    random.seed(seed)
    def batch(dataset, user_feature=user_feature, max_seq_len=max_seq_len):

        while True:
            for user_id in dataset.warm_users():
                input_npy = np.zeros(1, dtype=[('seq_item_id', (np.int32,  max_seq_len)),
                                               ('seq_len', np.int32),
                                               ('user_gender', np.int32),
                                               ('user_geo', np.int32)])

                item_list = dataset.get_positive_items(user_id, sort=sort)
                if len(item_list) <= 1:
                    continue
                train_items = item_list[-max_seq_len-1:-1]
                pad_train_items = np.zeros(max_seq_len, np.int32)
                pad_train_items[:len(train_items)] = train_items
                input_npy[0] = (pad_train_items,
                                len(train_items),
                                user_feature[user_id]['user_gender'],
                                user_feature[user_id]['user_geo'])
                yield [train_items[-1]], input_npy
                yield [], []
            yield None, None

    s = Sampler(dataset=dataset, generate_batch=batch, num_process=1)

    return s

In [0]:
import tensorflow as tf
from openrec.modules.extractions import MultiLayerFC


def MLPSoftmax(user, item, seq_len, max_seq_len, dims, subgraph, item_bias=None, 
               extra=None, l2_reg=None, labels=None, dropout=None, train=None, 
               scope=None):

    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):

    
        seq_mask = tf.sequence_mask(seq_len, max_seq_len, dtype=tf.float32)
        item = tf.reduce_mean(item * tf.expand_dims(seq_mask, axis=2), axis=1)

        if user is not None:
            in_tensor = tf.concat([user, item], axis=1)
        else:
            in_tensor = tf.concat([item], axis=1)

        if extra is not None:
            in_tensor = tf.concat([in_tensor, extra], axis=1)

        if train:
            logits = MultiLayerFC(in_tensor=in_tensor,
                                 dims=dims,
                                 subgraph=subgraph,
                                 bias_in=True,
                                 bias_mid=True,
                                 bias_out=False,
                                 dropout_mid=dropout,
                                 l2_reg=l2_reg,
                                 scope='mlp_reg')
        else:
            logits = MultiLayerFC(in_tensor=in_tensor,
                                 dims=dims,
                                 subgraph=subgraph,
                                 bias_in=True,
                                 bias_mid=True,
                                 bias_out=False,
                                 l2_reg=l2_reg,
                                 scope='mlp_reg')

        if item_bias is not None:
            logits += item_bias

        if train:
            loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                           logits=logits)
            subgraph.register_global_loss(tf.reduce_mean(loss))
        else:
            subgraph.register_global_output(tf.squeeze(logits))

# Your task 
-  understand reuse and extend an exsiting recommender
-  fill in the placeholders in the implementation of the `YouTubeRec` function 
-  successfully run the experimental code with the recommender you just built. 

In [0]:
from openrec.recommenders import VanillaYouTubeRec
from openrec.modules.extractions import LatentFactor
import tensorflow as tf

def YouTubeRec(batch_size, user_dict, item_dict, dim_user_embed, dim_item_embed, 
        max_seq_len, l2_reg_embed=None, l2_reg_mlp=None, dropout=None, 
        init_model_dir=None, save_model_dir='DRR/', train=True, serve=False):

  
    rec = VanillaYouTubeRec(batch_size=batch_size,
                            dim_item_embed=dim_item_embed['id'], 
                            max_seq_len=max_seq_len, 
                            total_items=item_dict['id'],
                            l2_reg_embed=l2_reg_embed, 
                            l2_reg_mlp=l2_reg_embed, 
                            dropout=dropout, 
                             init_model_dir=init_model_dir,
                            save_model_dir=save_model_dir, 
                            train=train, 
                            serve=serve)
    

    #TODO: fill in variables v1 and v2
    v1 = 'FILL_IN_VARIABLE_NAME'
    v2 = 'FILL_IN_VARIABLE_NAME'
    @rec.traingraph.inputgraph.extend(outs=[v1, v2])
    def add_feature(subgraph):
        subgraph[v1] = tf.placeholder(tf.int32, shape=[batch_size], name=v1)
        subgraph[v2] = tf.placeholder(tf.int32, shape=[batch_size], name=v2)
       
        subgraph.update_global_input_mapping({v1: subgraph[v1],
                                              v2: subgraph[v2]
                                             })

        
    #TODO: fill in variables v1 and v2
    v1 = 'FILL_IN_VARIABLE_NAME'
    v2 = 'FILL_IN_VARIABLE_NAME'
    @rec.servegraph.inputgraph.extend(outs=[v1, v2])
    def add_feature(subgraph):
        subgraph[v1] = tf.placeholder(tf.int32, shape=[None], name=v1)
        subgraph[v2] = tf.placeholder(tf.int32, shape=[None], name=v2)

        subgraph.update_global_input_mapping({v1: subgraph[v1],
                                              v2: subgraph[v2]})
        
    
    #TODO: fill in variables v1, v2, v3
    v1 = 'FILL_IN_VARIABLE_NAME'
    v2 = 'FILL_IN_VARIABLE_NAME'
    v3 = 'FILL_IN_VARIABLE_NAME'
    @rec.traingraph.usergraph(ins=[v1, v2], outs=[v3])
    @rec.servegraph.usergraph(ins=[v1, v2], outs=[v3])
    def user_graph(subgraph):
        _, o1 = LatentFactor(l2_reg=l2_reg_embed,
                              shape=[user_dict[v1], dim_user_embed[v1]],
                              id_=subgraph[v1],
                              subgraph=subgraph,
                              init='normal',
                              scope=v1)

        _, o2 = LatentFactor(l2_reg=l2_reg_embed,
                             shape=[user_dict[v2], dim_user_embed[v2]],
                             id_=subgraph[v2],
                             subgraph=subgraph,
                             init='normal',
                             scope=v2)
        subgraph[v3] = tf.concat([o1, o2], axis=1)
    
    
    
    #TODO: fill in variables v1
    v1 = 'FILL_IN_VARIABLE_NAME'
    @rec.traingraph.interactiongraph(ins=[v1, 'seq_vec', 'seq_len', 'label'])
    def train_interaction_graph(subgraph):
        dim = sum(list(dim_user_embed.values())) + sum(list(dim_item_embed.values()))
        MLPSoftmax(user=subgraph[v1],
                   item=subgraph['seq_vec'],
                   seq_len=subgraph['seq_len'],
                   max_seq_len=max_seq_len,
                   dims=[dim, item_dict['item_id']],
                   l2_reg=l2_reg_mlp,
                   labels=subgraph['label'],
                   dropout=dropout,
                   train=True,
                   subgraph=subgraph,
                   scope='MLPSoftmax'
                  )
        
        

    #TODO: fill in variables v1
    v1 = 'FILL_IN_VARIABLE_NAME'
    @rec.servegraph.interactiongraph(ins=[v1, 'seq_vec', 'seq_len'])
    def serve_interaction_graph(subgraph):
        dim = sum(list(dim_user_embed.values())) + sum(list(dim_item_embed.values()))
        MLPSoftmax(user=subgraph[v1],
                   item=subgraph['seq_vec'],
                   seq_len=subgraph['seq_len'],
                   max_seq_len=max_seq_len,
                   dims=[dim, item_dict['item_id']],
                   l2_reg=l2_reg_mlp,
                   train=False,
                   subgraph=subgraph,
                   scope='MLPSoftmax') 
    
    
    #TODO: fill in variables v1, v2, v3
    v1 = 'FILL_IN_VARIABLE_NAME'
    v2 = 'FILL_IN_VARIABLE_NAME'
    v3 = 'FILL_IN_VARIABLE_NAME'
    @rec.traingraph.connector.extend
    @rec.servegraph.connector.extend
    def connect(graph): 
        graph.usergraph[v1] = graph.inputgraph[v1]
        graph.usergraph[v2] = graph.inputgraph[v2]
        graph.interactiongraph[v3] = graph.usergraph[v3]

    return rec

# Experiement
We will use the recommender you implemented to run a toy experiement on the LastFM dataset. 

## load lastfm dataset

In [0]:
import numpy as np

train_data = np.load('lastfm_train.npy')
test_data = np.load('lastfm_test.npy')
user_feature = np.load('user_feature.npy')

total_users = 992   
total_items = 14598
user_dict = {'gender': 3, 
             'geo': 67}
item_dict = {'id': total_items}

In [0]:
user_feature[:10], test_data[:10]

## preprocessing dataset

In [0]:
from openrec.utils import Dataset

train_dataset = Dataset(train_data, total_users, total_items, 
                        sortby='ts', name='Train')
test_dataset = Dataset(test_data, total_users, total_items, 
                       sortby='ts', name='Test')

## hyperparameters and training parameters

In [0]:
dim_user_embed = {'geo': 40,    # dimension of user geographic embedding
                  'gender': 10, # dimension of user gender embedding
                   'total': 50} 
dim_item_embed = {'id': 50, 'total': 50}     # dimension of item embedding


max_seq_len = 100       # the maxium length of user's listen history
total_iter = int(1e3)   # iterations for training 
batch_size = 100        # training batch size
eval_iter = 100         # iteration of evaluation
save_iter = eval_iter   # iteration of saving model   

## define sampler
We use `DRRSampler`  and `DRREvaluationSampler` to sample sequences of training and testing samples. 

In [0]:
from openrec.utils.samplers import YouTubeSampler, YouTubeEvaluationSampler
  
train_sampler = YouTubeSampler(user_feature=user_feature, 
                                batch_size=batch_size, 
                                max_seq_len=max_seq_len, 
                                dataset=train_dataset, 
                                num_process=1)
test_sampler = YouTubeEvaluationSampler(user_feature=user_feature, 
                              dataset=test_dataset, 
                               max_seq_len=max_seq_len)

## define evaluator

In [0]:
from openrec.utils.evaluators import AUC, Recall

auc_evaluator = AUC()
recall_evaluator = Recall(recall_at=[100, 200, 300, 400, 500])

## define model trainer

we used the Vanilla version of the Youtube recommender to train our model.

In [0]:
from openrec import ModelTrainer
from openrec.recommenders import YouTubeRec

model = YouTubeRec(batch_size=batch_size,
                  user_dict=user_dict,
                  item_dict=item_dict,
                  max_seq_len=max_seq_len,
                  dim_item_embed=dim_item_embed,
                  dim_user_embed=dim_user_embed,
                  save_model_dir='youtube_recommender/',
                  train=True, serve=True)

model_trainer = ModelTrainer(model=model)

## training and testing

In [0]:
model_trainer.train(total_iter=total_iter, 
                    eval_iter=eval_iter,
                    save_iter=save_iter,
                    train_sampler=train_sampler,
                    eval_samplers=[test_sampler], 
                    evaluators=[auc_evaluator, recall_evaluator])